<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=a0b49d8d35fb263065db37f59bb63e5e306cd115085f12034e320a0f37b15ff0
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1


    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-06 10:41:17
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.24 C
CY Investment:  1.51 C
Reserve:  57.44 K
Current:  1.41 C
-------------------
Today PnL: -16.35 K (-0.12%)
Current PnL: -20.56 L (-13.65%)
CY Booked + Current PnL: -9.12 L (-6.06%)
-------------------
Total profit:  1.44 L
Total loss:  -22.00 L
-------------------
Total Booked + Current PnL: 17.86 L (14.44%)
Total Booked PnL: 38.42 L (31.06%)
Curr Year Booked PnL: 11.44 L (8.11%)
Prev Year Booked PnL: 26.98 L (21.81%)
Est FTT:  2.29 C
Est FTT PnL: 87.77 L (62.24%)
Deployed:  1.24 C
Current:  1.41 C
CAGR/XIRR %: 7.96%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-41.18,68.0,H-MC,13.47,116944.0,16440.0,12665.0,-0.23,16.36,10.83,28.96,79.0,1.30,0.83,55.88,MH,ATH,METALS
77,TTKPRESTIG,770.00,93.60,39.0,M-SC,1.53,83382.0,-17395.0,17485.0,-1.16,-17.26,20.97,0.09,245.0,-0.99,0.59,9.54,OX40N,NTT,DURABLES
14,BLUESTARCO,2080.00,9.78,49.0,H-MC,2.85,189570.0,24900.0,18426.0,-0.18,15.12,9.72,26.31,89.0,1.35,1.35,24.31,X40N,NTT,AC
42,ITC,452.00,-39.19,42.0,H-LC,2.04,195991.0,-4147.0,25028.0,-0.87,-2.07,12.77,10.44,4.0,-0.17,1.40,3.38,X40,NTT,FMCG
34,ICICIGI,2260.25,-17.56,57.0,H-MC,1.77,141843.0,5710.0,25418.0,0.27,4.19,17.92,22.86,91.0,0.22,1.01,17.34,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,RAJOOENG,143.10,-40.28,44.0,H-SC,16.07,90540.0,-11960.0,52558.0,5.00,-11.67,58.05,39.61,114.0,-0.23,0.64,10.58,AR,ATH,MISC
48,LTIM,7230.20,-4.91,52.0,H-LC,1.98,236092.0,-10059.0,89266.0,2.47,-4.09,37.81,32.18,16.0,-0.11,1.68,30.48,X200,ATH,IT
4,ANGELONE,3033.00,4.59,49.0,H-SC,11.29,184910.0,-6096.0,63794.0,2.44,-3.19,34.50,30.21,157.0,-0.10,1.32,17.02,X40N,NTT,FINANCE
78,UJJIVANSFB,60.00,111.20,69.0,H-SC,12.93,133191.0,-9288.0,28809.0,2.05,-6.52,21.63,13.70,163.0,-0.32,0.95,57.20,OX40N,NTT,BANKS
74,TCS,4406.34,-30.29,41.0,H-LC,12.27,277798.0,-68158.0,136399.0,1.84,-19.70,49.10,19.73,1.0,-0.50,1.98,2.32,X40,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAMMAANCAP,326.00,-181.82,63.0,M-SC,11.47,141723.0,-7497.0,151672.0,-4.58,-5.02,107.02,96.62,208.0,-0.05,1.01,53.37,XY25,NTT,FINANCE
66,SIS,528.00,2018.13,33.0,H-SC,3.60,85489.0,-25543.0,49148.0,-2.34,-23.01,57.49,21.26,156.0,-0.52,0.61,15.50,OX40N,NTT,MISC
82,VALIANTORG,838.00,-302.74,39.0,H-SC,3.38,112625.0,-50980.0,172294.0,-1.98,-31.16,152.98,74.15,139.0,-0.30,0.80,44.37,XR,NTT,CHEMICALS
13,BERGEPAINT,680.00,-19.75,49.0,H-MC,3.46,215662.0,-11677.0,59738.0,-1.86,-5.14,27.70,21.14,106.0,-0.20,1.54,21.56,XY24,NTT,PAINTS
71,TANLA,1963.11,-18.28,57.0,H-SC,19.12,218851.0,-51910.0,381851.0,-1.26,-19.17,174.48,121.86,133.0,-0.14,1.56,71.20,AR,ATH,IT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,ICICIPRULI,790.0,-21.11,48.0,H-MC,2.13,135668.0,-120.0,42871.0,-0.13,-0.09,31.6,31.48,107.0,-0.0,0.97,12.02,X40,ATH,INSURANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,111.20,69.0,H-SC,12.93,133191.0,-9288.0,28809.0,2.05,-6.52,21.63,13.70,163.0,-0.32,0.95,57.20,OX40N,NTT,BANKS
45,KANSAINER,340.0,-67.42,56.0,H-SC,2.00,224721.0,-44946.0,81282.0,-1.06,-16.67,36.17,13.47,138.0,-0.55,1.60,14.26,XY24,NTT,PAINTS
77,TTKPRESTIG,770.0,93.60,39.0,M-SC,1.53,83382.0,-17395.0,17485.0,-1.16,-17.26,20.97,0.09,245.0,-0.99,0.59,9.54,OX40N,NTT,DURABLES
18,CERA,9475.0,-22.91,37.0,H-SC,1.47,140530.0,-35373.0,77390.0,-0.63,-20.11,55.07,23.89,149.0,-0.46,1.00,21.37,OX40N,NTT,CERAMICS
72,TATAELXSI,9161.0,-24.05,44.0,H-MC,6.21,102704.0,-29131.0,71359.0,0.93,-22.10,69.48,32.03,98.0,-0.41,0.73,15.04,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1263.64,71.0,M-SC,20.22,185447.0,4304.0,41169.0,0.51,2.38,22.20,25.10,235.0,0.10,1.32,48.62,XY24,NTT,HEALTHCARE
60,SAIL,228.00,47.55,60.0,M-MC,12.35,232814.0,7852.0,159804.0,0.18,3.49,68.64,74.53,192.0,0.05,1.66,36.87,XY24,BTT,STEEL
32,HINDZINC,730.22,31.71,69.0,M-LC,5.60,214498.0,9422.0,103152.0,0.33,4.59,48.09,54.89,52.0,0.09,1.53,29.80,X5K,ATH,METALS
14,BLUESTARCO,2080.00,9.78,49.0,H-MC,2.85,189570.0,24900.0,18426.0,-0.18,15.12,9.72,26.31,89.0,1.35,1.35,24.31,X40N,NTT,AC
84,VOLTAS,1530.00,-3.32,44.0,H-MC,1.62,202905.0,11163.0,26601.0,-0.57,5.82,13.11,19.69,99.0,0.42,1.44,13.02,XY25,NTT,AC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-41.18,68.0,H-MC,13.47,116944.0,16440.0,12665.0,-0.23,16.36,10.83,28.96,79.0,1.30,0.83,55.88,MH,ATH,METALS
11,BANKINDIA,190.00,-25.47,69.0,H-MC,10.18,193219.0,13411.0,99005.0,0.08,7.46,51.24,62.52,88.0,0.14,1.38,44.09,XR,NTT,BANKS
32,HINDZINC,730.22,31.71,69.0,M-LC,5.60,214498.0,9422.0,103152.0,0.33,4.59,48.09,54.89,52.0,0.09,1.53,29.80,X5K,ATH,METALS
37,INDIAMART,4810.62,-55.20,33.0,H-SC,9.49,125186.0,1850.0,129780.0,-0.40,1.50,103.67,106.72,119.0,0.01,0.89,25.23,AR,ATH,MISC
86,WIPRO,420.00,-15.15,41.0,M-LC,5.66,149395.0,-1550.0,111000.0,-0.01,-1.03,74.30,72.51,53.0,-0.01,1.06,4.93,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-22.15,30.0,H-LC,9.78,280560.0,-35282.0,147939.0,-0.83,-11.17,52.73,35.67,5.0,-0.24,2.00,1.14,X40N,ATH,FMCG
5,ASIANPAINT,3465.66,-18.03,33.0,H-LC,5.81,206923.0,-44845.0,98061.0,-0.27,-17.81,47.39,21.13,27.0,-0.46,1.47,11.30,X40,ATH,PAINTS
57,RELIANCE,1533.00,-14.79,41.0,H-LC,3.39,213096.0,2730.0,26062.0,0.19,1.30,12.23,13.68,37.0,0.10,1.52,18.04,XY25,NTT,REFINERIES
74,TCS,4406.34,-30.29,41.0,H-LC,12.27,277798.0,-68158.0,136399.0,1.84,-19.70,49.10,19.73,1.0,-0.50,1.98,2.32,X40,ATH,IT
42,ITC,452.00,-39.19,42.0,H-LC,2.04,195991.0,-4147.0,25028.0,-0.87,-2.07,12.77,10.44,4.0,-0.17,1.40,3.38,X40,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.5,-0.90,51.0,H-LC,1.87,159140.0,-26955.0,74430.0,0.59,-14.48,46.77,25.51,15.0,-0.36,1.13,17.27,AR,ATH,ELECTRICAL
48,LTIM,7230.2,-4.91,52.0,H-LC,1.98,236092.0,-10059.0,89266.0,2.47,-4.09,37.81,32.18,16.0,-0.11,1.68,30.48,X200,ATH,IT
42,ITC,452.0,-39.19,42.0,H-LC,2.04,195991.0,-4147.0,25028.0,-0.87,-2.07,12.77,10.44,4.0,-0.17,1.40,3.38,X40,NTT,FMCG
51,NESTLEIND,1377.0,-10.90,48.0,H-LC,3.19,274388.0,8962.0,47826.0,0.23,3.38,17.43,21.40,11.0,0.19,1.95,10.24,XY25,NTT,FMCG
57,RELIANCE,1533.0,-14.79,41.0,H-LC,3.39,213096.0,2730.0,26062.0,0.19,1.30,12.23,13.68,37.0,0.10,1.52,18.04,XY25,NTT,REFINERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-22.15,30.0,H-LC,9.78,280560.0,-35282.0,147939.0,-0.83,-11.17,52.73,35.67,5.0,-0.24,2.00,1.14,X40N,ATH,FMCG
74,TCS,4406.34,-30.29,41.0,H-LC,12.27,277798.0,-68158.0,136399.0,1.84,-19.70,49.10,19.73,1.0,-0.50,1.98,2.32,X40,ATH,IT
42,ITC,452.00,-39.19,42.0,H-LC,2.04,195991.0,-4147.0,25028.0,-0.87,-2.07,12.77,10.44,4.0,-0.17,1.40,3.38,X40,NTT,FMCG
30,HCLTECH,1937.36,-2.80,43.0,H-LC,10.01,221904.0,-20020.0,82260.0,1.43,-8.28,37.07,25.73,8.0,-0.24,1.58,4.86,X40,ATH,IT
40,INFY,2275.00,-20.32,44.0,H-LC,8.15,311086.0,-1802.0,173493.0,0.96,-0.58,55.77,54.87,3.0,-0.01,2.21,6.04,X40,BTT,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,-0.90,51.0,H-LC,1.87,159140.0,-26955.0,74430.0,0.59,-14.48,46.77,25.51,15.0,-0.36,1.13,17.27,AR,ATH,ELECTRICAL
42,ITC,452.00,-39.19,42.0,H-LC,2.04,195991.0,-4147.0,25028.0,-0.87,-2.07,12.77,10.44,4.0,-0.17,1.40,3.38,X40,NTT,FMCG
19,CIPLA,1795.00,-22.52,42.0,H-LC,4.00,206541.0,2041.0,39367.0,-0.67,1.00,19.06,20.25,10.0,0.05,1.47,10.04,X40N,BTT,PHARMA
5,ASIANPAINT,3465.66,-18.03,33.0,H-LC,5.81,206923.0,-44845.0,98061.0,-0.27,-17.81,47.39,21.13,27.0,-0.46,1.47,11.30,X40,ATH,PAINTS
57,RELIANCE,1533.00,-14.79,41.0,H-LC,3.39,213096.0,2730.0,26062.0,0.19,1.30,12.23,13.68,37.0,0.10,1.52,18.04,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4406.34,-30.29,41.0,H-LC,12.27,277798.0,-68158.0,136399.0,1.84,-19.70,49.10,19.73,1.0,-0.50,1.98,2.32,X40,ATH,IT
40,INFY,2275.00,-20.32,44.0,H-LC,8.15,311086.0,-1802.0,173493.0,0.96,-0.58,55.77,54.87,3.0,-0.01,2.21,6.04,X40,BTT,IT
42,ITC,452.00,-39.19,42.0,H-LC,2.04,195991.0,-4147.0,25028.0,-0.87,-2.07,12.77,10.44,4.0,-0.17,1.40,3.38,X40,NTT,FMCG
83,VBL,671.64,-22.15,30.0,H-LC,9.78,280560.0,-35282.0,147939.0,-0.83,-11.17,52.73,35.67,5.0,-0.24,2.00,1.14,X40N,ATH,FMCG
1,ABB,7934.00,-40.16,48.0,H-LC,7.57,250152.0,-11467.0,130679.0,0.54,-4.38,52.24,45.57,7.0,-0.09,1.78,6.44,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-45.67,46.0,L-SC,32.87,80429.0,-33120.0,73174.0,1.60,-29.17,90.98,35.27,268.0,-0.45,0.57,96.85,XR,NTT,HOTELS
49,MASFIN,398.61,-20.05,40.0,H-SC,11.42,90690.0,-7290.0,28894.0,-0.49,-7.44,31.86,22.05,152.0,-0.25,0.65,31.82,XR,ATH,FINANCE
50,NATIONALUM,244.55,-41.18,68.0,H-MC,13.47,116944.0,16440.0,12665.0,-0.23,16.36,10.83,28.96,79.0,1.30,0.83,55.88,MH,ATH,METALS
78,UJJIVANSFB,60.00,111.20,69.0,H-SC,12.93,133191.0,-9288.0,28809.0,2.05,-6.52,21.63,13.70,163.0,-0.32,0.95,57.20,OX40N,NTT,BANKS
61,SAMMAANCAP,326.00,-181.82,63.0,M-SC,11.47,141723.0,-7497.0,151672.0,-4.58,-5.02,107.02,96.62,208.0,-0.05,1.01,53.37,XY25,NTT,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
69,SURYODAY,240.00,66.67,70.0,H-SC,6.02,153510.0,-25561.0,91292.0,-0.99,-14.27,59.47,36.71,135.0,-0.28,1.09,52.31,XR,NTT,BANKS
61,SAMMAANCAP,326.00,-181.82,63.0,M-SC,11.47,141723.0,-7497.0,151672.0,-4.58,-5.02,107.02,96.62,208.0,-0.05,1.01,53.37,XY25,NTT,FINANCE
64,SHALBY,327.00,1263.64,71.0,M-SC,20.22,185447.0,4304.0,41169.0,0.51,2.38,22.20,25.10,235.0,0.10,1.32,48.62,XY24,NTT,HEALTHCARE
32,HINDZINC,730.22,31.71,69.0,M-LC,5.60,214498.0,9422.0,103152.0,0.33,4.59,48.09,54.89,52.0,0.09,1.53,29.80,X5K,ATH,METALS
78,UJJIVANSFB,60.00,111.20,69.0,H-SC,12.93,133191.0,-9288.0,28809.0,2.05,-6.52,21.63,13.70,163.0,-0.32,0.95,57.20,OX40N,NTT,BANKS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.99
1,25,43.24
2,50,74.88


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.21
LC    32.03
MC    23.76
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.95
X40      14.53
X40N     11.22
XR       11.02
XY25     10.32
AR        8.20
OX40N     7.77
X200      1.68
MH        1.63
X5K       1.53
SR        1.15
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.07
H-LC    25.49
H-MC    20.49
M-SC    13.71
M-LC     5.53
M-MC     2.97
L-SC     1.43
L-LC     1.01
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.79,-6.02,40.96
IT,12.60,-18.64,82.89
FINANCE,10.45,-15.08,67.36
MISC,6.95,-17.70,82.92
BANKS,6.41,-10.90,69.24
PAINTS,5.56,-18.33,35.97
ELECTRICAL,5.32,-10.01,48.60
INSURANCE,3.76,-4.92,44.15
AUTO,3.53,-15.82,71.19


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3229046.0,22
XR,1360988.0,14
AR,1176852.0,9
X40,800061.0,10
X40N,629941.0,9
OX40N,555577.0,10
XY25,488092.0,7
SR,259876.0,2
X5K,103152.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3707645.0,29
M-SC,1507736.0,17
H-LC,1230730.0,15
H-MC,1228752.0,15
M-LC,373101.0,4
M-MC,365839.0,2
L-SC,260287.0,3
L-MC,59841.0,1
L-LC,43018.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1218057.0      6
M-SC       XY24       845456.0      7
H-SC       AR         804610.0      5
           XR         789364.0      7
H-MC       XY24       569575.0      4
H-LC       X40        515241.0      5
M-MC       XY24       365839.0      2
H-SC       X40N       308696.0      4
           SR         259876.0      2
           OX40N      255609.0      4
H-LC       X40N       237171.0      3
H-MC       X40        211445.0      4
H-LC       AR         205109.0      2
H-MC       XY25       180629.0      2
L-SC       XR         171571.0      2
M-SC       AR         167133.0      2
M-LC       XY24       158949.0      2
M-SC       XY25       151672.0      1
           OX40N      139893.0      4
           XR         130207.0      2
H-LC       XY25       112773.0      3
M-LC       XR         111000.0      1
           X5K        103152.0      1
H-MC       XR          99005.0      1
H-LC       X200        89266.0      1
L-SC       OX40N       88716.0      1
H-MC       X40N        84074.0      2
M-SC       X40         73375.0      1
H-SC       MH          71433.0      1
H-MC       OX40N       71359.0      1
H-LC       XY24        71170.0      1
L-MC       XR          59841.0      1
L-LC       XY25        43018.0      1
H-MC       MH          12665.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 31.0 seconds
